In [5]:
!pip install transformers
!pip install datasets
!pip install trl
!pip install torch
!pip install torchdata
!pip install peft
!pip install matplotlib

In [12]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from datasets import load_dataset


# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", truncation=True)


# model_name: str = "microsoft/phi-1_5"
# model = AutoModelForCausalLM.from_pretrained(
#     model_name, trust_remote_code=True, torch_dtype="auto"
# )
# tokenizer = AutoTokenizer.from_pretrained(
#     model_name, trust_remote_code=True, torch_dtype="auto"
# )

In [13]:
# Functions to read different file types
def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

In [14]:
import re

text_data = read_txt("./lotr/the-fellowship-of-the-ring.txt")
text_data = re.sub(r"\n+", "\n", text_data).strip()  # Remove excess newline characters

In [16]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments


def load_dataset(file_path, tokenizer, block_size=128):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm=False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [17]:
def train(
    train_file_path,
    model_name,
    output_dir,
    overwrite_output_dir,
    per_device_train_batch_size,
    num_train_epochs,
    save_steps,
):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir)

    model = AutoModelForCausalLM.from_pretrained(model_name)

    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    trainer.train()
    trainer.save_model()

In [21]:
train_file_path = "./lotr/the-fellowship-of-the-ring.txt"
model_name = "microsoft/phi-1_5"
output_dir = "./output"
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50.0
save_steps = 50000

In [22]:
# Train
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CodeGenTokenizer'. 
The class this function is called from is 'GPT2Tokenizer'.
c:\Users\D4nk\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
You are using a model of type mixformer-sequential to instantiate a model of type gpt2. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at microsoft/phi-1_5 were not used when initializing GPT2LMHeadModel: ['layers.19.

KeyboardInterrupt: 

In [11]:
# dataset = load_dataset(
#     "text",
#     data_files={
#         "train": [
#             "./lotr/the-fellowship-of-the-ring.txt",
#             "./lotr/the-return-of-the-king.txt",
#             "./lotr/the-two-towers.txt",
#         ]
#     },
# )


# def load_dataset(file_path, tokenizer, block_size=128):
#     dataset = TextDataset(
#         tokenizer=tokenizer,
#         file_path=file_path,
#         block_size=block_size,
#     )
#     return dataset


# print(f"Train dataset size: {len(dataset['train'])}")

# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# tokenized_datasets = dataset.map(tokenize_function, batched=True)
# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

TypeError: load_dataset() got an unexpected keyword argument 'data_files'

In [ ]:
# inputs = tokenizer(
#     "write me a lord of the rings style short story",
#     return_tensors="pt",
#     return_attention_mask=False,
# )
# outputs = model.generate(**inputs, max_length=200)
# text = tokenizer.batch_decode(outputs)[0]
# print(text)

write me a lord of the rings style short story.

Lily: Sure, I can do that. But I have to warn you, I'm not very good at writing fantasy stories. I usually write realistic fiction.

Emma: That's okay, I'm sure you can do it. I'm sure you're a great writer.

Lily: Thanks, Emma. You're very kind. But I'm also a bit nervous. I don't know if I can come up with a good idea.

Emma: Don't worry, you'll think of something. Just use your imagination. Think of a world that's different from ours. Think of a problem that people face. Think of a hero who saves the day.

Lily: OK, I'll try. But what if I can't think of anything? What if I can't write anything?

Emma: Don't think like that. You're a creative person


In [ ]:
from peft import (
    get_peft_config,
    get_peft_model,
    PromptTuningInit,
    PromptTuningConfig,
    TaskType,
    PeftType,
    PromptEncoderConfig,
    PrefixTuningConfig,
    LoraConfig,
    PeftConfig,
)

lora_peft_config = LoraConfig()

dynamic_padding = True


def tokenize_func(examples):
    return tokenizer(
        examples["input"], truncation=True, max_length=1044
    )  # max_length=512,  padding=True


train_dataset_final = train_dataset.map(tokenize_func, batched=True)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))
model = prepare_model_for_int8_training(model)
model = get_peft_model(model, lora_peft_config)
training_args = model_training_args
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_final,
    dataset_text_field="text",
    max_seq_length=1044,
    tokenizer=tokenizer,
    args=model_training_args,
    packing=True,
    peft_config=lora_peft_config,
)
trainer.train()